In [2]:
from collections import deque

def unify(term1, term2):
    """
    Unifies two terms, returning a substitution dictionary if unification is possible,
    or raising an exception if unification fails.
    """
    substitution = {}
    worklist = deque([(term1, term2)])

    while worklist:
        t1, t2 = worklist.popleft()

        if t1 == t2:
            continue

        elif is_variable(t1):
            if t1 in substitution:
                worklist.append((substitution[t1], t2))
            else:
                substitution[t1] = t2

        elif is_variable(t2):
            if t2 in substitution:
                worklist.append((t1, substitution[t2]))
            else:
                substitution[t2] = t1

        elif isinstance(t1, tuple) and isinstance(t2, tuple):
            if t1[0] == t2[0]:
                worklist.extend(zip(t1[1:], t2[1:]))
            else:
                raise Exception(f"Cannot unify {t1} with {t2}")

        else:
            raise Exception(f"Cannot unify {t1} with {t2}")

    return substitution

def is_variable(term):
    """Checks if the term is a variable (starts with a lowercase letter)"""
    return isinstance(term, str) and term.islower()

def apply_substitution(substitution, term):
    """Applies a substitution to a term"""
    if term in substitution:
        return apply_substitution(substitution, substitution[term])
    elif isinstance(term, tuple):
        return (term[0],) + tuple(apply_substitution(substitution, arg) for arg in term[1:])
    else:
        return term

def parse_term(term_str):
    """Parses a term from a string input (e.g., 'f(x, g(y))') into a Python tuple."""
    term_str = term_str.strip()
    if '(' not in term_str:
        return term_str  # Return as string if it's a constant or variable
    func_name = term_str.split('(')[0]
    args_str = term_str[len(func_name)+1:-1]
    args = [parse_term(arg.strip()) for arg in args_str.split(',')]
    return (func_name,) + tuple(args)

# Main function to take user input
def main():
    try:
        # Take user input for two terms to unify
        print("Enter first term (e.g., f(x, g(y)))")
        term1_str = input("Term 1: ")
        term1 = parse_term(term1_str)

        print("Enter second term (e.g., f(x, g(z)))")
        term2_str = input("Term 2: ")
        term2 = parse_term(term2_str)

        # Perform unification
        substitution = unify(term1, term2)
        print("Unification successful. Substitutions:", substitution)

        # Apply the substitution and show the unified term
        unified_term = apply_substitution(substitution, term1)
        print("Unified term:", unified_term)

    except Exception as e:
        print("Unification failed:", e)

# Run the program
if __name__ == "__main__":
    main()


Enter first term (e.g., f(x, g(y)))
Term 1: f(x,g(y))
Enter second term (e.g., f(x, g(z)))
Term 2: f(x, g(z))
Unification successful. Substitutions: {'y': 'z'}
Unified term: ('f', 'x', ('g', 'z'))
